In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
# 指定当前的工作文件夹
import os
# 此处为google drive中的文件路径,drive为之前指定的工作根目录，要加上
os.chdir("/content/drive/My Drive/LapSRN_myModel") 

## 训练

In [ ]:
import os
os.chdir("/content/drive/My Drive/LapSRN_myModel") 

!python "train.py" --dataset "../DataSet_less/" --nEpochs 20 --cuda  --batchSize 1

## 测试

In [3]:
!python "test.py" \
--input '/content/drive/My Drive/TestImg/val/95-512pix-speed7-ave1.tif' \
--model_r '/content/drive/My Drive/TestImg/LapSRN_r_epoch_300_No_Perception.pth' \
--model_g '/content/drive/My Drive/TestImg/LapSRN_g_epoch_300_No_Perception.pth' \
--outputHR2 "/content/drive/My Drive/TestImg/result/X2/output95_X2_epoch_300_NoPerception.tif" \
--outputHR4 "/content/drive/My Drive/TestImg/result/X4/output95_X4_epoch_300_NoPerception.tif" \
--cuda

Namespace(cuda=True, input='/content/drive/My Drive/TestImg/val/95-512pix-speed7-ave1.tif', model_g='/content/drive/My Drive/TestImg/LapSRN_g_epoch_300_No_Perception.pth', model_r='/content/drive/My Drive/TestImg/LapSRN_r_epoch_300_No_Perception.pth', outputHR2='/content/drive/My Drive/TestImg/result/X2/output95_X2_epoch_300_NoPerception.tif', outputHR4='/content/drive/My Drive/TestImg/result/X4/output95_X4_epoch_300_NoPerception.tif')
saved !


## 编写`test.py`,现在按照上面测试即可

In [ ]:
import os
from torchvision.transforms import Compose, ToTensor, CenterCrop
os.chdir("/content/drive/My Drive/LapSRN_myModel/") 

from __future__ import print_function
import argparse
import torch
from torch.autograd import Variable
from PIL import Image
from torchvision.transforms import Compose, ToTensor, Lambda
import torchvision
import numpy as np


# model = torch.load(opt.model, map_location='cpu')
model_r = torch.load('/content/drive/My Drive/LapSRN_myModel/LapSRN__r_epoch_450.pth')
model_g = torch.load('/content/drive/My Drive/LapSRN_myModel/LapSRN_g_epoch_450.pth')
model_r = model_r.cuda()
model_g = model_g.cuda()

transform = Compose(
    [ToTensor(),
         ])

img = Image.open("/content/drive/My Drive/TestImg/val/44-512pix-speed7-ave1.tif").convert('RGB')
r, g, _ = img.split()

r = transform(r)
r = r.unsqueeze(0) 
r = r.cuda()

g = transform(g)
g = g.unsqueeze(0) 
g = g.cuda()

HR_2_r, HR_4_r = model_r(r)
HR_2_g, HR_4_g = model_g(g)
# HR_2_r = HR_2_r.cpu()
# HR_4_r = HR_4_r.cpu()
black_2 = torch.zeros(1, 1024, 1024).unsqueeze(0).cuda()
HR_2 = torch.cat((HR_2_r.squeeze(0), HR_2_g.squeeze(0), black_2.squeeze(0))).unsqueeze(0)
black_4 = torch.zeros(1, 2048, 2048).unsqueeze(0).cuda()
HR_4 = torch.cat((HR_4_r.squeeze(0), HR_4_g.squeeze(0), black_4.squeeze(0))).unsqueeze(0)
torchvision.utils.save_image(HR_2, 'out_test_x2_%d.tif'% (1029002), padding=0) # 输出保存
torchvision.utils.save_image(HR_4, 'out_test_x4_%d.tif'% (1029002), padding=0) # 输出保存

In [ ]:
out_HR_2_r = HR_2_r.squeeze(0).detach().numpy()
print(out_HR_2_r.shape)
out_HR_2_r *= 255.0
out_HR_2_r = out_HR_2_r.clip(0, 255)
print(out_HR_2_r.shape)
out_HR_2_r = np.transpose(out_HR_2_r,(1, 2, 0))
print(out_HR_2_r.shape)
out_HR_2_r = Image.fromarray(np.uint8(out_HR_2_r), mode='RGB')

(3, 1024, 1024)
(3, 1024, 1024)
(1024, 1024, 3)


In [ ]:
out_HR_4_r = HR_4_r.squeeze(0).detach().numpy()
print(out_HR_4_r.shape)
out_HR_4_r *= 255.0
out_HR_4_r = out_HR_4_r.clip(0, 255)
print(out_HR_4_r.shape)
out_HR_4_r = np.transpose(out_HR_4_r,(1, 2, 0))
print(out_HR_4_r.shape)
out_HR_4_r = Image.fromarray(np.uint8(out_HR_4_r), mode='RGB')

(3, 2048, 2048)
(3, 2048, 2048)
(2048, 2048, 3)


In [ ]:
out_HR_2_r.save('/content/drive/My Drive/TestImg/LapSRN_RGB_ID44_Epoch400_X2_1023_2.tif')
out_HR_4_r.save('/content/drive/My Drive/TestImg/LapSRN_RGB_ID44_Epoch400_X4_1023_2.tif')

In [ ]:
import torchvision
HR_2_r, HR_4_r = model(LR_r)  # 模型输出
torchvision.utils.save_image(HR_2_r.data,'test_x2_%d.tif'% (102503), padding=0) # 输出保存
torchvision.utils.save_image(HR_4_r.data,'test_x4_%d.tif'% (102503), padding=0) # 输出保存

## 用于释放Cuda显存

In [ ]:
!apt install psmisc

Reading package lists... Done
Building dependency tree       
Reading state information... Done
psmisc is already the newest version (23.1-1ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 11 not upgraded.


In [ ]:
!sudo fuser /dev/nvidia*

In [ ]:
!kill -9 123

/bin/bash: line 0: kill: (123) - No such process
